# examples for `hostess.aws.ec2`

### introduction

`hostess.aws.ec2` is a collection of utilities for working with EC2 instances.

`Instance` and `Cluster` are its centerpiece classes. They are abstractions for, 
respectively, single EC2 instances and groups of EC2 instances. By offering both
managed interaction with the EC2 API and a rich set of remote procedure call (RPC)
capabilities, they attempt to make distributed workflows as conceptually simple and 
immediate as local ones. 

### requirements

1.  You need appropriate AWS permissions to perform any actions that call the
    EC2 API. You cannot, for instance, use `ls_instances()` without the ListInstances
    permission, or `Instance.start()` without StartInstances permission for the
    particular instance you are attempting to start. (A complete discussion of AWS
    permissions management is beyond the scope of this document.)

    By default, `hostess` uses the 'default' profile from ~/.aws/credentials.
    This can be modified in `hostess.config.user_config` or by manually constructing
    a client with `hostess.aws.utils.init_client`. 

    If you are in a situation where you have SSH access to, but not AWS permissions for, 
    an EC2 instance you'd like to make `hostess` RPCs on, you should ignore the fact that
    it is an EC2 instance and simply use `hostess.ssh.SSH`, which underlies the RPC
    capabilities of this module.
    
3. The RPC functionality offered by this module relies on SSH, so you need SSH access to an
   instance to make RPCs on it. Specifically:
   
    1. The inbound traffic rules of the instance's security group must permit SSH access
       from your IP. (If you create a security group with `hostess`, it will set this up
       automatically.)
    2. `hostess` supports only keyfile-based authentication, so you must have the correct
      keyfile for the instance, and `hostess` must be able to find it. If it shares a
      filename with the key name known to AWS and is in your `~/.ssh` folder, `hostess`
      will find it automatically. If this is not the case, you can manually specify a
      path to the keyfile when constructing an `Instance` or `Cluster`. You can also change
      the default search paths in `hostess.config.user_config`. (If you create a security
      group with `hostess`, it will also generate a compatible keyfile and save it into
      your `~/.ssh` folder.) 
    3. The instance must be running `sshd` and configured to accept incoming connections.
      You generally will not have to do anything special to set this up: the default
      configuration of most stock AMIs, including the Ubuntu and Amazon Linux images, is
      suitable.

   *note: if you want to create remote workflows without relying on SSH connections, we recommend
   looking at the `hostess.station` framework.* 

### relationship to `boto3`

`aws.ec2.Instance` is designed to be easier to use, more Pythonic, and more powerful than `boto3`'s 
`Instance` abstraction. However, it does not implement high-level interfaces for the entirety 
of the EC2 API. Because it works partly by wrapping parts of `boto3`, however, every `aws.ec2.Instance` 
also grants access to a `boto3` `Instance` object with a shared AWS configuration. If 
you need access to other parts of the EC2 API inside a `hostess` workflow, you can access this object
via the `aws.ec2.Instance.instance_` attribute.

## 1. listing and finding instances

*note: the output of these examples won't be very interesting if you don't currently
have any instances. if you don't have any, but you do have an AWS profile configured on 
your computer, you might want to skip down to the 'launching instances' section.*

If you have ever used the `awscli` command `aws ec2 describe-instances` to try to find
an EC2 instance, you may have noticed that although its output is very complete, it is
extremely verbose, deeply nested, and hard to parse. Its filtering options are also 
somewhat difficult to use. `hostess`'s `ls_instances()` is a much more lightweight 
alternative that is equally suitable for most use cases.

In [ ]:
# if you simply call `ls_instances()` with no arguments, it will return a tuple
# of dicts giving essential information about all your running, pending, stopping,
# or stopped instances:
# name (if any); public ip (if any); instance id; state (running, 
# pending, stopped, terminated); private ip (if any), and keyname (if any). 
from hostess.aws.ec2 import ls_instances

instance_info = ls_instances()
instance_info[0:2]

In [ ]:
# ls_instances() offers a variety of ways to filter your search.
# see the docstring for a full set of options; we'll just describe
# one here. ls_instances() understands arbitrary keywords arguments 
# as case-insensitive tag filters. 
# if you have a single instance named 'kitty', you can find it with:
ls_instances(name='kitty')[0]

In [ ]:
# this feature also supports optional regex matching. if you have a 
# set of instances named 'pipeline_1', 'pipeline_2', etc.:
ls_instances(name=r'pipeline_\d', tag_regex=True)

## 2. launching instances

`Instance` can be used both to work with existing instances and launch new ones.
In this section, we'll launch an instance to work with in subsequent sections.
We should discuss some preliminaries first.

**IMPORTANT**: `hostess` performs no automated instance lifecycle management
due to its potential to cause process disruption and unintended data loss. 
If you want to stop or terminate an instance, you must do so explicitly,
using methods of the `Instance` object, other interfaces to the EC2 API, shutdown
commands on the instance itself, or the EC2 web console. 

### 2a. instance configuration

#### launch templates

Although it's not mandatory, the easiest and cleanest way to launch instances with 
`hostess` is to use an existing launch template and give `Instance.launch()` the name
of the template.
(On the backend, this is because the API requires a launch template to launch an 
instance, and reusing the same launch template helps ensure consistent behavior).
You can create a launch template using `aws.ec2.create_launch_template()`, 
other interfaces to the EC2 API, or the EC2 web console. 
**TODO: template parameter overrides are a planned feature. update this when finished.** 

If you don't do this, `hostess` just creates a 'scratch' template to launch the 
instance and deletes it immediately after launch (whether successful or failed).

#### defaults

The EC2 API requires explicit specification of a lot of parameters to launch an instance.
`hostess` wants to make it easy to launch instances, so if you don't explicitly
specify some or all of these parameters, it populates them with sensible
defaults. You can override the following defaults in `hostess.aws.config.user_config`:
* instance_type (instance type, like 't3a.micro')
* volume_type (root EBS volume type, like 'gp3')
* volume_size (root EBS volume size in GB)

**TODO: complete description of valid `options`**

A couple of default behaviors are worth special consideration:

* If you don't specify a security group, either in a launch template or
  the 'security_group_name' item of `options`, `hostess` acts like
  the EC2 web console's launch wizard and creates a new
  security group.
  * These auto-generated security groups are named 'hostess-'
    followed by 10 random lowercase letters. They permit all outgoing traffic
    from their members, and permit incoming traffic on:
    * the standard SSH port (22) from your current IP
    * any port from other AWS resources in the same security group.
  * this also generates an SSH keyfile and saves it to your ~/.ssh folder. Its
    filename is the same as the name of its associated security group.
  * `hostess` does **not** provide automated lifecycle management for these security
    groups (this is because deleting security groups associated with running
    instances is impossible, and deleting security groups associated with stopped
    instances can make them unusable).
    This has the same issue as the EC2 launch wizard: it can leave your account
    cluttered with old security groups, which are harmless but unsightly. It can
    also leave your .ssh folder confusingly cluttered with keyfiles. It's
    better to reuse existing security groups.
* If you don't specify an Amazon Machine Image (AMI), either in a launch template
  or in the 'image_id' item of `options`, `hostess` uses the most recent
  Ubuntu Server LTS AMI.

### 2b. executing `Instance.launch()`

The `hostess` defaults are permissive enough that you can make a new instance
with default configuration simply by calling `Instance.launch()` with no arguments. 
However, we'll show a preferred workflow here.

In [ ]:
# first, create a reusable launch template with a few non-default options:
from hostess.aws.ec2 import create_launch_template

response = create_launch_template(
    template_name='kitty',
    # a cheap burstable instance
    instance_type='t3a.micro',
    volume_size=9,
    # gp3 volumes typically have better sustained performance, and hostess
    # defaults to gp3. However, gp2 volumes tend to have better initial 
    # startup and shutdown times, so they're better for a quick demo.
    volume_type='gp2',
    instance_name='kitty',
    tags={'BillCode': 'Customer5', 'Project': 'ADMIN'}
)

# note that you can't have multiple templates with the same name.
# if you happen to have run this cell previously and want to create a 
# new 'kitty' template with different settings, you'll need to delete
# the former template, like: 
# from hostess.aws.ec2 import init_client
# init_client('ec2').delete_launch_template(LaunchTemplateName='kitty')

In [ ]:
# now launch an instance using the template. you can specify the 'kitty' 
# template in subsequent launches to create identically-configured instances 
# in the same security group.

from hostess.aws.ec2 import Instance

# We're passing connect=True here to make sure the instance is fully
# booted up before we send the stop instruction in the next section --
# if you send a stop instruction to a booting instance, it can take
# a really long time to stop.
kitty = Instance.launch(template='kitty', connect=True)

# 3. controlling instance state

Instance has several methods to control an instance's activation state:
* `start()` boots the instance.
* `stop()` shuts the instance down.
* `restart()` shuts the instance down, waits for complete shutdown, and boots it again.
* `terminate()` terminates the instance.
    **--IMPORTANT--**: this permanently and irrevocably deletes an instance and, unless it's
    specifically configured otherwise, its root volume. `hostess` trusts that you know
    what you're doing, so `terminate()` doesn't have any special guardrails.
    `Instance.terminate()` is like `sudo rm -rf`: don't even type it unless you really mean it!

#### notes:
* These methods don't do anything if the instance is already in (or transitioning to) the requested state.
* `start()`, `stop()`, and `restart()` will raise exceptions on terminated instances.
* `Instance()` also has `wait_until_running()`, `wait_until_stopped()`, and
  `wait_until_terminated()` methods, which block until the instance reaches the specified state. 

In [ ]:
# for the sake of demonstration in the next section, shut the 'kitty'
# instance down and delete the Instance object from the namespace. 
# note that deleting the Instance object does _not_ terminate the actual 
# instance; you'll need to terminate it later if you don't want it to 
# hang around in your account.
kitty.stop()
# wait until kitty fully shuts down. may take a few minutes.
kitty.wait_until_stopped() 
del kitty

## 4. connecting to an existing instance
 
`Instance` can connect to an existing instance even
more easily than it can launch a new one.

The only required argument to the `Instance` constructor is an
identifier for the EC2 instance you'd like to work with. There are three
acceptable types of identifier:

* a connectable IP address for the instance, like `"102.31.4.129"`
* the AWS instance identifier, like `"i-0868ad3eeebe16cde"`
* one of the `dicts` returned by `ls_instances()`

*note: if you're connecting to an instance from another EC2 instance, 
pass `use_private_ip=True` to the `Instance` constructor, and if 
you're using an IP address as the identifier in this case, make sure
it's the private IP.*

In [ ]:
# kitty exists, but is stopped, and we don't have an Instance object
# for it. we can find kitty and construct a new Instance for it
# using Instance along with ls_instances():
from hostess.aws.ec2 import Instance, ls_instances

kitty = Instance(ls_instances(name='kitty')[0])

# the string representation of an Instance gives its name (if any),
# its instance id, its instance type, its EC2 subnet, and its IP address
# (if it has one, which it generally won't if it's not running). 
print(kitty)

# Instance also has a number of 'basic information' attributes, like::
print(
    f"the instance is {kitty.state}, named '{kitty.name}', "
    f"and has the following tags:\n{kitty.tags}"
)

In [ ]:
# let's boot kitty back up so we can use it.
# it can take a minute or two for an instance, especially a little 
# instance like kitty, to launch its SSH daemon after it boots. 
# connect=True makes start() block until you can establish an SSH
# connection to the instance -- here, we're doing this to ensure
# that we'll be able to run commands in the next section.
kitty.start(connect=True)

# now it's got an ip!
kitty

## 4. remote procedure calls

`Instance` supports two main types of remote procedure calls (RPCs):

* shell commands
* Python function calls

### 4a. shell commands

With `Instance`, you can run commands as if you were logged 
into the instance and work with the output of those commands in Python. 
`Instance` has three primary methods for this. They are all highly 
configurable, and we don't discuss all their options here. See the 
documentation for `hostess.subutils.RunCommand` and 
`hostess.subutils.Viewer` for a full description of options.

* `command()` runs a command in the remote user's default login shell.
  you can pass a command as a literal string, or construct it from
  Python arguments (examples below). By default it runs the command
  asynchronously and returns a `hostess.subutils.Viewer` object you
  can use to inspect or terminate the process. 
* `con()` simulates the experience of typing a command into a console
  and looking at its output. It blocks until the process exits and
  pretty-prints any output from the process.
    * This means that if you run a command that never exits, like
    `'while sleep 1; do uptime; done'`, `con()` will block until
    it is interrupted or the connection is severed. 
    * `con()` prints "^C" on `KeyboardInterrupt` rather than raising
    the exception.
* `commands()` provides syntactic sugar for constructing list commands.
  
*note: `hostess` only fully supports `bash`. some functionality may not work in other shells.*

In [ ]:
# get the full UNIX name information for the instance.
# `hostess` interprets the a=True argument as a shell 
# switch; this is equivalent to kitty.command('uname -a').
uname = kitty.command('uname', a=True)

# because Instance.command() runs asynchronously by default,
# it's unlikely that you'll get any output in the microseconds
# it takes to get to this next line. you'll probably just see 
# that 'uname' is a Viewer for a running 'uname -a' process. 
uname

In [ ]:
# it'll probably have completed by the time you execute
# this cell, though (and if not, just run the cell again).
uname

In [ ]:
# if you want a more streamlined way to look at the output of a command,
# try Instance.con(). blocks until process exit and iteratively 
# pretty-prints its stdout and stderr (stderr in red).
# It's intended to give the feel of running a command in a console.
# IMPORTANT: by default, in order to keep things nice and light, con() 
# does not return anything. If you need to use the results of the command 
# in subsequent code, pass _return_viewer=True.

# to pretty-print all active TCP connections, timestamped, twice:
kitty.con("date; ss -t ; sleep 1; date ; ss -t")

In [ ]:
# if you'd like to ensure that a command completes before you move on 
# to the next part of your code, but don't want it to be loud like con(),
# you can call the .wait() method of the returned Viewer object or pass 
# _wait=True:
usage = kitty.command("df -h", _wait=True)
print(usage.stdout[0])

In [ ]:
# you can access the stdout and stderr of commands you execute
# via the .out and .err attributes of returned Viewers.

# these are lists of strings. each element of the lists is an 
# individual write to stdout/stderr by the remote process. 
# simple commands that do a thing and exit will generally only have one element, 
# because they return their output all at once. (this may not be
# true in cases in which the output is extremely large, due to SSH 
# buffering, etc.
print(f"{len(usage.out)} write(s) to stdout")

In [ ]:
# this allows you to use the results of remote shell commands in code.
import re
import pandas as pd

rows = [
    re.split(' +', line, maxsplit=5) 
    for line in usage.out[0].splitlines()
]
pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
# it also allows you to monitor the results of ongoing processes.
# a silly example:
kitty.command("echo 1 > numbers.txt", _wait=True)
tail = kitty.command("tail -f numbers.txt")
number = 1
while len(tail.out) < 5:
    number += 1
    kitty.command(f"echo {number} >> numbers.txt", _wait=True)
    print(tail.out)

In [ ]:
# it can sometimes be important to do manual cleanup of backgrounded RPCs. 
# the tail process is still running, even though we're done with it:
tail.running

In [ ]:
# so you might want to kill it.
tail.kill()
tail.running

In [ ]:
# Instance.commands() is an easy way to perform a sequence of commands
# without having to enter a monolithic string. a silly example:
kitty.commands(["cd /", "ls"], _wait=True)

In [ ]:
# this can be used for serious sysadmin stuff. 
# a real-world example might look like this:
from itertools import chain

private_repos = ["sensitive_devops", "proprietary_algos", "company_secrets"]
update_commands = chain(
    *[
        (f"ssh-add .ssh/{repo}_deploy", f"cd {repo}", "git pull", "cd ~")
        for repo in private_repos
    ]
)
update_result = kitty.commands(
    ["eval `ssh-agent`", *update_commands], op="and", _wait=True
)

# this won't actually work, of course, because these are 
# hypothetical keys and repos, but you get the idea.
print(f"the directories don't really exist:\n{update_result.err[0]}")

# note that op="and" caused hostess to chain the long sequence of commands
# with &&, which is why bash didn't continue after `ssh-add` failed:
print(f"\nwe tried to run\n{update_result.command}")

### 4b. python function calls

`Instance.call_python()` permits direct invocation of Python
functions in any installed interpreter.

#### interpreter selection and installation

`call_python()` requires a Python interpreter on the remote host. 
* By default, it uses the first `python` on the shell $PATH (if any). 
* You can explicitly specify the path to an interpreter by passing it
  as the `interpreter_path` argument.
* If you want to run code in a conda env, you can pass the name of
  the env as the `env` argument, and `hostess` will find it for you.

The stock Ubuntu AWS image doesn't come with Python.
We'd like to demonstrate the special relationship between 
`call_python()` and Conda, so we're going to install
the Miniforge distribution of Python.

In [ ]:
from pathlib import Path
miniforge_url = (
    "https://github.com/conda-forge/miniforge/releases/latest/"
    "download/Miniforge3-Linux-x86_64.sh"
)
# this might take a minute or two.
installation = kitty.commands(
    [f"wget {miniforge_url}", f"sh {Path(miniforge_url).name} -b"],
    op="and",
)
installation.wait()
print(installation.out[-1])

#### making function calls

`call_python()` lets you directly call Python functions on an instance,
from simple math functions to top-level handlers for complex applications. 
A silly example:

In [ ]:
from more_itertools import chunked

# roll 3d6 6 times
rolls = [
    kitty.call_python(
        "random", "randint", (1, 6), splat="*", env="base"
    )
    for die in range(18)
]
# the processes are running asynchronously; wait for them to complete
[r.wait() for r in rolls]
 
# chunk and sum the results
rolls = chunked([int(r.out[0]) for r in rolls], 3)
stats = [
    kitty.call_python(None, "sum", roll, env="base") for roll in rolls
]
[s.wait() for s in stats]
[int(s.out[0]) for s in stats]

#### a note on performance

It is perfectly appropriate to call computationally inexpensive 
procedures with `call_python()` for utility, administration, and 
similar purposes.

However, if you're doing intense computation, `call_python()` is best-used 
to call functions that act as entry points to larger pipelines, or at least 
perform larger units of work. The preceding example is an exaggeratedly 
inefficient use of `call_python()`, sending 24 separate networked instructions
to and executing 24 separate Python interpreter processes to add a few random numbers. 
For simple calculations like this, the overhead involved in making the remote calls
and launching the interpreter far outweighs the program execution itself.

----

Now, a less silly example: quickly turn kitty into a simple web proxy.

In [ ]:
# write a simple script to retrieve web content
fetch_script = """
import requests

def fetch_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text
"""
# write the script to the instance. literal_str=True means:
# write fetch_script into fetch.py as a string instead
# of trying to interpret fetch_script as a path to a file.
kitty.put(fetch_script, "fetch.py", literal_str=True)

def fetch(url):
    response = kitty.call_python(
        "/home/ubuntu/fetch.py", "fetch_url", url, env="base", _wait=True
    )
    return ''.join(response.out)

In [ ]:
from IPython.display import HTML

# now you can read Shakespeare in perfect privacy.
HTML(fetch("http://shakespeare.mit.edu/midsummer/midsummer.3.1.html"))

#### limitations of `call_python()`

* `call_python()` is versatile but is not designed for passing enormous 
single arguments. If you want to use `call_python()` in applications that 
work with large quantities of data, you'll be better off storing the 
data in files and calling functions that accept their filenames as 
arguments.
* `call_python()` does not currently implement automated serialization or
compression for return arguments. If you want to get serialized objects
back from the instance, you will need to ensure that the called
function includes a serialization step and then deserialize it yourself
locally. this may change in the future.
    * *note: if you want to dynamically pass Python objects around in a
       supervised way, consider looking at the `hostess.station()`
      framework.*

### 4b.1: remote Jupyter Notebook access

Many Python programmers (including us) suffer from a deep attachment to Jupyter.
Firing off scripts is lovely, but sometimes you need to roll up your sleeves
and dig around in an interactive environment, and while shells are fine, cell 
persistence, graphical capabilities, and ... well, we probably don't need to
talk you into it.

On the other hand, setting up remote Jupyter servers can be a giant hassle, 
and if you permit HTTP access to them, it's very hard to make them secure.
Fortunately, `Instance` includes a streamlined method for launching a Jupyter
server and tunneling to it over SSH: `Instance.notebook()`. It's a Notebook there, 
but here, and just for you.

In [ ]:
# kitty doesn't have Jupyter, so we'll need to install it first.
# you should expect this to take 2-3 minutes depending on internet 
# weather -- Jupyter Notebook has a lot of dependencies.

kitty.con(
    "/home/ubuntu/miniforge3/bin/mamba create -y -n notebook notebook"
)

In [ ]:
# there are no mandatory arguments to Instance.notebook(),
# although it'll only work that way if you have Jupyter 
# installed in a Python environment on your default $PATH.
# we'll show some useful options here for the sake of
# demonstration:

url, tunnel, tunnel_info, launch, stopper = kitty.notebook(
    # what port would you like to use for the local end
    # of the tunnel? it's useful to be able to change this 
    # if you want to run multiple Instance-powered notebooks
    # or have a lot of other port forwarding going on.
    # also, if you shut down the Notebook but don't kill your local
    # tunnel (see below) you won't be able to bind anything else
    # to that port.
    # automated tunnel closing is a planned feature, but it 
    # won't be perfectly reliable in all cases.
    # defaults to 22222.
    local_port=12358,
    # what port would you like the instance to launch 
    # Jupyter Server on? defaults to 8888, the default 
    # Jupyter Server port.
    remote_port=10001,
    # optional name of a conda environment -- if not specified,
    # just tries to run a jupyter executable from the $PATH.
    env='notebook',
    # what directory would you like Jupyter to use as 
    # its root working directory? if not specified, defaults 
    # to the remote user's home directory, so the value here
    # here is redundant and just for the sake of demonstration.
    # this is useful because it's difficult to 'back up' from
    # the directory Jupyter launches in, so you won't be
    # able to access anything outside your home directory 
    # tree by default.
    working_directory='/home/ubuntu',
    # set this to True to use JupyterLab instead of Notebook
    lab=False
)
# navigate to the url and try out kitty's Notebook:
url

#### tips for `Instance.notebook()`

1. `notebook()` launches Jupyter daemonized. This means 
   that if you lose your connection for whatever reason,
   whatever Jupyter might have been doing on the instance
   will not stop. You can even reestablish a tunnel to
   the instance and continue working.
    
   It also means that if you kill the launch process 
   (`launch` in the example above), it won't stop Jupyter 
   in the way hitting CTRL-C does when you run it in an interactive 
   session. `notebook()`, however, returns a function that you can call 
   to gracefully shut down Jupyter (`stopper` in the 
   example above). If something's gone very wrong with Jupyter
   and this doesn't work, take the gloves off and run
   `Instance.command('killall jupyter-noteboo')`. Also, if you 
   *want* Jupyter to shut down when you disconnect,
   you can pass `kill_on_exit=True` to `notebook()`, although
   this isn't guaranteed to work if your local Python process
   segfaults or something.
2. `notebook()` also returns a Process object encapsulating 
   the SSH tunnel, along with metadata about the tunnel (`tunnel` 
   and `tunnel_info` in the example above). If something
   goes wrong with the tunnel, inspecting these can be useful.
3. Finally, `notebook()` returns a `Viewer` for the process that 
   actually launched Jupyter (`launch` in the example above). 
   Examining this can be useful as a diagnostic for Jupyter-level 
   issues. Unless you pass `kill_on_exit=True`, killing this process 
   won't actually kill Jupyter; it'll just keep you from receiving 
   its console output.
4. Conventions for controlling Jupyter have changed with Notebook 7.
   We do not guarantee backwards compatibility with earlier versions
   of Notebook.

### 4c. managing SSH connections

* as you've seen, you normally don't have to manually establish an SSH connection to
an instance. it will happen automatically when you try to use `Instance` functionality that
requires a connection. However, the `connect()` method can be used to ensure that you 
_can_ connect to the instance, or to 'prep' it so that there's no connection delay 
on your first command when you get to that part of a program.
    * this method does nothing if a connection is already open.
* you might also want to close an existing connection and open a new one -- for
instance, if you lost track of a bunch of processes or the connection gets externally
disrupted in a weird way. you can do this with the `reconnect()` method.
    * this will immediately terminate all non-daemonized processes executed
over the exiting connection and close any established tunnels / forwarded ports.
    * if no connection is currently open, `reconnect()` is equivalent to `connect()`.
* By default, EC2 uses a strict IP allowlist to gate incoming traffic to instances. 
This provides extremely good security, and `hostess` mimics this behavior by default. However,
this means that if your IP address changes, you won't be able to connect to your instances
anymore until you modify their security group settings. This can be a big hassle if you're 
on a cell network, using a VPN with IP hopping, or simply moving around a lot.
`Instance.rebase_ssh_ingress_ip()` can quickly solve this issue for you: it modifies the 
security groups associated with an instance to permit SSH access from your IP.
    * **IMPORTANT:** this method revokes all other ingress permissions. Do not use
    it if you need an instance to be accessible to anyone or anything but you. This may change in the future.
* If you'd like to establish an SSH tunnel on a specific port so that you can access some service on the instance in another application, you can call `tunnel(local_port, remote_port)`. For instance, if you launched a Notebook on the instance using the earlier cell, and then dropped connection for some reason, you could reestablish the tunnel with `Instance.tunnel(12358, 10001)` and go back to whatever you were doing in Notebook. Same deal for anything from PostgreSQL (5432) to DOOM (666).

## 9999. clusters

### 9999a. creating a `Cluster`

Because on-demand instance launch requests are actually 'fleet requests' on the backend, `Instance.launch()` 
is actually a thin wrapper around `Cluster.launch()`, so just refer to the earlier section on launching instances
for a detailed description of most of `Cluster.launch()`'s behavior. The only difference in the signature is 
that `Cluster.launch` requires a `count` argument specifying the number of instances to launch.